In [1]:
import os
import csv
import pandas as pd


import pathlib
import torch

from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from typing import Tuple, Dict, List



In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
#https://github.com/mrdbourke/pytorch-deep-learning/blob/main/04_pytorch_custom_datasets.ipynb
from torch.utils.data import Dataset
import numpy as np

class SignalCustom(Dataset):
    
    def __init__(self, csv: str) -> None:
        self.df = pd.read_csv(csv, sep='\t')

        self.classes, self.class_to_idx = (['norm', 'mi'], {'norm': 0, 'mi': 1})

    
    def __len__(self) -> int:
        "Returns the total number of samples."
        return len(self.df)
    

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        "Returns one sample of data, data and label (X, y)."
        st =self.df.at[index, 'signal'].replace('[', '')
        st = st.replace(']', '')
        st = st.split()
        st = [float(i) for i in st]
        signal = torch.unsqueeze(torch.from_numpy(np.array(st)), 0)
        
        class_name  = self.df.at[index, 'mi_class'] 
        class_idx = int(class_name)
        return signal, class_idx # return data, label (X, y)

In [79]:
true_data = SignalCustom(csv='./RESULTS/datasets/true-s.csv')
conv_data = SignalCustom(csv='./RESULTS/datasets/conv-s.csv')
conv_inc_data = SignalCustom(csv='./RESULTS/datasets/conv-inc-s.csv')

true_data_test = SignalCustom(csv='./RESULTS/datasets/true-s-test.csv')
conv_data_test = SignalCustom(csv='./RESULTS/datasets/conv-s-test.csv')
conv_inc_data_test = SignalCustom(csv='./RESULTS/datasets/conv-inc-s-test.csv')

In [80]:
from torch.utils.data import DataLoader
true_dl = DataLoader(dataset=true_data, batch_size=64, num_workers=0, shuffle=True) 
conv_dl = DataLoader(dataset=conv_data, batch_size=64, num_workers=0, shuffle=True)
conv_inc_dl = DataLoader(dataset=conv_inc_data, batch_size=64, num_workers=0, shuffle=True)

true_dl_test = DataLoader(dataset=true_data_test, batch_size=64, num_workers=0) 
conv_dl_test = DataLoader(dataset=conv_data_test, batch_size=64, num_workers=0)
conv_inc_dl_test = DataLoader(dataset=conv_inc_data_test, batch_size=64, num_workers=0)

In [10]:
for sign, cls in true_dl:
    print(sign.shape, cls)
    break

torch.Size([64, 1, 400]) tensor([0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
        0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0])


In [83]:
import torch.nn as nn
class ClassificationBase(nn.Module):
    def training_step(self, batch):
        prices, labels = batch 
        prices = prices.to(device, dtype=torch.float)
        out = self(prices)        
        
        labels = labels.to(device)
        loss = F.cross_entropy(out, labels) # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy

        return loss, acc
    
    def validation_step(self, batch):
        prices, labels = batch 
        prices = prices.to(device, dtype=torch.float)
      
        labels = labels.to(device)

        out = self(prices)     
        out = out.to(device)
        
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [84]:
class TimeSeriesCnnModel(ClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2), # output: 64 x 125

            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2), # output: 128 x 62

            nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2), # output: 256 x 31

            nn.Flatten(), 
            nn.Linear(12800, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 2))
        
    def forward(self, xb):
        return self.network(xb)

In [85]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [86]:
model1 = TimeSeriesCnnModel()
model1.to(device)
    
from torchinfo import summary

summary(model1, input_size=[1, 1, 400]) # do a test pass through of an example input size 

Layer (type:depth-idx)                   Output Shape              Param #
TimeSeriesCnnModel                       [1, 2]                    --
├─Sequential: 1-1                        [1, 2]                    --
│    └─Conv1d: 2-1                       [1, 32, 400]              128
│    └─ReLU: 2-2                         [1, 32, 400]              --
│    └─Conv1d: 2-3                       [1, 64, 400]              6,208
│    └─ReLU: 2-4                         [1, 64, 400]              --
│    └─MaxPool1d: 2-5                    [1, 64, 200]              --
│    └─Conv1d: 2-6                       [1, 128, 200]             24,704
│    └─ReLU: 2-7                         [1, 128, 200]             --
│    └─Conv1d: 2-8                       [1, 128, 200]             49,280
│    └─ReLU: 2-9                         [1, 128, 200]             --
│    └─MaxPool1d: 2-10                   [1, 128, 100]             --
│    └─Conv1d: 2-11                      [1, 256, 100]             98,560

In [150]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        train_acc = []
        for batch in train_loader:
            loss, acc = model.training_step(batch)
            train_losses.append(loss)
            train_acc.append(acc)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()        
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [151]:
device = get_default_device()
true_dl = DeviceDataLoader(true_dl, device)
true_dl_test = DeviceDataLoader(true_dl_test, device)
to_device(model1, device);

In [152]:
num_epochs = 1010
opt_func = torch.optim.Adam
lr = 0.00001

In [153]:
import torch.nn.functional as F
model1 = TimeSeriesCnnModel()
model1.to(device)
to_device(model1, device);
history = fit(num_epochs, lr, model1, true_dl, true_dl_test, opt_func)

Epoch [0], train_loss: 0.6932, val_loss: 0.6931, val_acc: 0.5312
Epoch [1], train_loss: 0.6933, val_loss: 0.6928, val_acc: 0.5312
Epoch [2], train_loss: 0.6933, val_loss: 0.6932, val_acc: 0.4703
Epoch [3], train_loss: 0.6933, val_loss: 0.6930, val_acc: 0.5312
Epoch [4], train_loss: 0.6931, val_loss: 0.6931, val_acc: 0.4990
Epoch [5], train_loss: 0.6931, val_loss: 0.6931, val_acc: 0.4932
Epoch [6], train_loss: 0.6931, val_loss: 0.6929, val_acc: 0.5312
Epoch [7], train_loss: 0.6930, val_loss: 0.6928, val_acc: 0.5312
Epoch [8], train_loss: 0.6929, val_loss: 0.6925, val_acc: 0.5312
Epoch [9], train_loss: 0.6922, val_loss: 0.6931, val_acc: 0.4948
Epoch [10], train_loss: 0.6905, val_loss: 0.6922, val_acc: 0.5146
Epoch [11], train_loss: 0.6874, val_loss: 0.6901, val_acc: 0.5474
Epoch [12], train_loss: 0.6857, val_loss: 0.6922, val_acc: 0.5130
Epoch [13], train_loss: 0.6846, val_loss: 0.6937, val_acc: 0.5099
Epoch [14], train_loss: 0.6827, val_loss: 0.6968, val_acc: 0.4969
Epoch [15], train_lo

Epoch [124], train_loss: 0.1570, val_loss: 1.8834, val_acc: 0.5432
Epoch [125], train_loss: 0.1453, val_loss: 1.9184, val_acc: 0.5771
Epoch [126], train_loss: 0.1428, val_loss: 1.9244, val_acc: 0.5562
Epoch [127], train_loss: 0.1433, val_loss: 2.0104, val_acc: 0.5615
Epoch [128], train_loss: 0.1349, val_loss: 2.0206, val_acc: 0.5568
Epoch [129], train_loss: 0.1304, val_loss: 2.1181, val_acc: 0.5458
Epoch [130], train_loss: 0.1303, val_loss: 2.1232, val_acc: 0.5651
Epoch [131], train_loss: 0.1316, val_loss: 2.1432, val_acc: 0.5391
Epoch [132], train_loss: 0.1248, val_loss: 2.1778, val_acc: 0.5547
Epoch [133], train_loss: 0.1174, val_loss: 2.2613, val_acc: 0.5490
Epoch [134], train_loss: 0.1286, val_loss: 2.2684, val_acc: 0.5193
Epoch [135], train_loss: 0.1299, val_loss: 2.3095, val_acc: 0.5417
Epoch [136], train_loss: 0.1219, val_loss: 2.4426, val_acc: 0.5417
Epoch [137], train_loss: 0.1171, val_loss: 2.4038, val_acc: 0.5583
Epoch [138], train_loss: 0.1050, val_loss: 2.4163, val_acc: 0.

Epoch [247], train_loss: 0.0023, val_loss: 5.8922, val_acc: 0.5370
Epoch [248], train_loss: 0.0024, val_loss: 5.9017, val_acc: 0.5432
Epoch [249], train_loss: 0.0025, val_loss: 5.9128, val_acc: 0.5385
Epoch [250], train_loss: 0.0021, val_loss: 5.9624, val_acc: 0.5370
Epoch [251], train_loss: 0.0020, val_loss: 5.9931, val_acc: 0.5385
Epoch [252], train_loss: 0.0019, val_loss: 5.9958, val_acc: 0.5401
Epoch [253], train_loss: 0.0018, val_loss: 6.0513, val_acc: 0.5469
Epoch [254], train_loss: 0.0018, val_loss: 6.0506, val_acc: 0.5333
Epoch [255], train_loss: 0.0018, val_loss: 6.1013, val_acc: 0.5448
Epoch [256], train_loss: 0.0016, val_loss: 6.1593, val_acc: 0.5437
Epoch [257], train_loss: 0.0016, val_loss: 6.1455, val_acc: 0.5359
Epoch [258], train_loss: 0.0015, val_loss: 6.2008, val_acc: 0.5354
Epoch [259], train_loss: 0.0014, val_loss: 6.2180, val_acc: 0.5354
Epoch [260], train_loss: 0.0014, val_loss: 6.2672, val_acc: 0.5417
Epoch [261], train_loss: 0.0014, val_loss: 6.3370, val_acc: 0.

Epoch [370], train_loss: 0.0003, val_loss: 7.1734, val_acc: 0.5432
Epoch [371], train_loss: 0.0003, val_loss: 7.1865, val_acc: 0.5406
Epoch [372], train_loss: 0.0003, val_loss: 7.1929, val_acc: 0.5323
Epoch [373], train_loss: 0.0003, val_loss: 7.2063, val_acc: 0.5276
Epoch [374], train_loss: 0.0003, val_loss: 7.2271, val_acc: 0.5323
Epoch [375], train_loss: 0.0003, val_loss: 7.2427, val_acc: 0.5354
Epoch [376], train_loss: 0.0002, val_loss: 7.3047, val_acc: 0.5359
Epoch [377], train_loss: 0.0003, val_loss: 7.2809, val_acc: 0.5354
Epoch [378], train_loss: 0.0002, val_loss: 7.3063, val_acc: 0.5292
Epoch [379], train_loss: 0.0002, val_loss: 7.3188, val_acc: 0.5354
Epoch [380], train_loss: 0.0002, val_loss: 7.3606, val_acc: 0.5417
Epoch [381], train_loss: 0.0002, val_loss: 7.3747, val_acc: 0.5385
Epoch [382], train_loss: 0.0002, val_loss: 7.3942, val_acc: 0.5385
Epoch [383], train_loss: 0.0002, val_loss: 7.4173, val_acc: 0.5391
Epoch [384], train_loss: 0.0002, val_loss: 7.4344, val_acc: 0.

Epoch [493], train_loss: 0.0000, val_loss: 9.6236, val_acc: 0.5328
Epoch [494], train_loss: 0.0000, val_loss: 9.6364, val_acc: 0.5354
Epoch [495], train_loss: 0.0000, val_loss: 9.6694, val_acc: 0.5354
Epoch [496], train_loss: 0.0000, val_loss: 9.6993, val_acc: 0.5328
Epoch [497], train_loss: 0.0000, val_loss: 9.7193, val_acc: 0.5328
Epoch [498], train_loss: 0.0000, val_loss: 9.7289, val_acc: 0.5312
Epoch [499], train_loss: 0.0000, val_loss: 9.7347, val_acc: 0.5339
Epoch [500], train_loss: 0.0000, val_loss: 9.7613, val_acc: 0.5323
Epoch [501], train_loss: 0.0000, val_loss: 9.7842, val_acc: 0.5328
Epoch [502], train_loss: 0.0000, val_loss: 9.8064, val_acc: 0.5281
Epoch [503], train_loss: 0.0000, val_loss: 9.8116, val_acc: 0.5339
Epoch [504], train_loss: 0.0000, val_loss: 9.8381, val_acc: 0.5354
Epoch [505], train_loss: 0.0000, val_loss: 9.8639, val_acc: 0.5328
Epoch [506], train_loss: 0.0000, val_loss: 9.8854, val_acc: 0.5328
Epoch [507], train_loss: 0.0000, val_loss: 9.9022, val_acc: 0.

Epoch [614], train_loss: 0.0000, val_loss: 11.8935, val_acc: 0.5323
Epoch [615], train_loss: 0.0000, val_loss: 11.9371, val_acc: 0.5250
Epoch [616], train_loss: 0.0000, val_loss: 11.9371, val_acc: 0.5339
Epoch [617], train_loss: 0.0000, val_loss: 11.9521, val_acc: 0.5292
Epoch [618], train_loss: 0.0000, val_loss: 11.9668, val_acc: 0.5307
Epoch [619], train_loss: 0.0000, val_loss: 11.9838, val_acc: 0.5281
Epoch [620], train_loss: 0.0000, val_loss: 12.0065, val_acc: 0.5281
Epoch [621], train_loss: 0.0000, val_loss: 12.0127, val_acc: 0.5323
Epoch [622], train_loss: 0.0000, val_loss: 12.0328, val_acc: 0.5307
Epoch [623], train_loss: 0.0000, val_loss: 12.0582, val_acc: 0.5297
Epoch [624], train_loss: 0.0000, val_loss: 12.0790, val_acc: 0.5307
Epoch [625], train_loss: 0.0000, val_loss: 12.0872, val_acc: 0.5276
Epoch [626], train_loss: 0.0000, val_loss: 12.1297, val_acc: 0.5281
Epoch [627], train_loss: 0.0000, val_loss: 12.1485, val_acc: 0.5266
Epoch [628], train_loss: 0.0000, val_loss: 12.16

Epoch [735], train_loss: 0.0000, val_loss: 13.9386, val_acc: 0.5297
Epoch [736], train_loss: 0.0000, val_loss: 13.9434, val_acc: 0.5312
Epoch [737], train_loss: 0.0000, val_loss: 13.9641, val_acc: 0.5281
Epoch [738], train_loss: 0.0000, val_loss: 13.9837, val_acc: 0.5312
Epoch [739], train_loss: 0.0000, val_loss: 13.9896, val_acc: 0.5312
Epoch [740], train_loss: 0.0000, val_loss: 14.0079, val_acc: 0.5281
Epoch [741], train_loss: 0.0000, val_loss: 14.0148, val_acc: 0.5292
Epoch [742], train_loss: 0.0000, val_loss: 14.0347, val_acc: 0.5266
Epoch [743], train_loss: 0.0000, val_loss: 14.0492, val_acc: 0.5328
Epoch [744], train_loss: 0.0000, val_loss: 14.0594, val_acc: 0.5297
Epoch [745], train_loss: 0.0000, val_loss: 14.0772, val_acc: 0.5266
Epoch [746], train_loss: 0.0000, val_loss: 14.1010, val_acc: 0.5297
Epoch [747], train_loss: 0.0000, val_loss: 14.1078, val_acc: 0.5297
Epoch [748], train_loss: 0.0000, val_loss: 14.1215, val_acc: 0.5266
Epoch [749], train_loss: 0.0000, val_loss: 14.13

Epoch [856], train_loss: 0.0000, val_loss: 12.8272, val_acc: 0.5490
Epoch [857], train_loss: 0.0000, val_loss: 12.8261, val_acc: 0.5490
Epoch [858], train_loss: 0.0000, val_loss: 12.8268, val_acc: 0.5505
Epoch [859], train_loss: 0.0000, val_loss: 12.8268, val_acc: 0.5505
Epoch [860], train_loss: 0.0000, val_loss: 12.8261, val_acc: 0.5505
Epoch [861], train_loss: 0.0000, val_loss: 12.8260, val_acc: 0.5505
Epoch [862], train_loss: 0.0000, val_loss: 12.8253, val_acc: 0.5505
Epoch [863], train_loss: 0.0000, val_loss: 12.8260, val_acc: 0.5505
Epoch [864], train_loss: 0.0000, val_loss: 12.8262, val_acc: 0.5505
Epoch [865], train_loss: 0.0000, val_loss: 12.8256, val_acc: 0.5505
Epoch [866], train_loss: 0.0000, val_loss: 12.8260, val_acc: 0.5505
Epoch [867], train_loss: 0.0000, val_loss: 12.8255, val_acc: 0.5505
Epoch [868], train_loss: 0.0000, val_loss: 12.8254, val_acc: 0.5505
Epoch [869], train_loss: 0.0000, val_loss: 12.8258, val_acc: 0.5474
Epoch [870], train_loss: 0.0000, val_loss: 12.82

Epoch [977], train_loss: 0.0000, val_loss: 12.8279, val_acc: 0.5505
Epoch [978], train_loss: 0.0000, val_loss: 12.8277, val_acc: 0.5505
Epoch [979], train_loss: 0.0000, val_loss: 12.8271, val_acc: 0.5505
Epoch [980], train_loss: 0.0000, val_loss: 12.8291, val_acc: 0.5505
Epoch [981], train_loss: 0.0000, val_loss: 12.8287, val_acc: 0.5521
Epoch [982], train_loss: 0.0000, val_loss: 12.8289, val_acc: 0.5521
Epoch [983], train_loss: 0.0000, val_loss: 12.8289, val_acc: 0.5521
Epoch [984], train_loss: 0.0000, val_loss: 12.8291, val_acc: 0.5521
Epoch [985], train_loss: 0.0000, val_loss: 12.8299, val_acc: 0.5521
Epoch [986], train_loss: 0.0000, val_loss: 12.8298, val_acc: 0.5521
Epoch [987], train_loss: 0.0000, val_loss: 12.8312, val_acc: 0.5521
Epoch [988], train_loss: 0.0000, val_loss: 12.8305, val_acc: 0.5521
Epoch [989], train_loss: 0.0000, val_loss: 12.8304, val_acc: 0.5521
Epoch [990], train_loss: 0.0000, val_loss: 12.8305, val_acc: 0.5536
Epoch [991], train_loss: 0.0000, val_loss: 12.83

In [ ]:
torch.save({
    "epoch": num_epochs,
    "model1_state_dict": model1.state_dict(),
}, "./RESULTS/model-true-1.pth")

In [ ]:
device = get_default_device()
conv_dl = DeviceDataLoader(conv_dl, device)
conv_dl_test = DeviceDataLoader(conv_dl_test, device)
model2 = TimeSeriesCnnModel()
model2.to(device)
to_device(model2, device);
history = fit(num_epochs, lr, model2, conv_dl,conv_dl_test, opt_func)

In [ ]:
torch.save({
    "epoch": num_epochs,
    "model2_state_dict": model2.state_dict(),
}, "./RESULTS/model-conv-1.pth")

In [ ]:
device = get_default_device()
conv_inc_dl = DeviceDataLoader(conv_inc_dl, device)
conv_inc_dl_test = DeviceDataLoader(conv_inc_dl_test, device)
model3 = TimeSeriesCnnModel()
model3.to(device)
to_device(model3, device);
history = fit(num_epochs, lr, model3, conv_inc_dl, conv_inc_dl_test, opt_func)

In [ ]:
torch.save({
    "epoch": num_epochs,
    "model3_state_dict": model3.state_dict(),
}, "./RESULTS/model-inc-1.pth")

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [139]:
model1.eval()
model2.eval()
model3.eval()

TimeSeriesCnnModel(
  (network): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (3): ReLU()
    (4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (6): ReLU()
    (7): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (8): ReLU()
    (9): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (11): ReLU()
    (12): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (13): ReLU()
    (14): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(start_dim=1, end_dim=-1)
    (16): Linear(in_features=12800, out_features=1024, bias=True)
    (17): ReLU()
    (18): Linear(in_features=1024, out_features=512, bia

In [154]:
# train: true, test: true
evaluate(model1, true_dl_test)

{'val_loss': 12.838408470153809, 'val_acc': 0.5520833730697632}

In [155]:
# train: true, test: conv
evaluate(model1, conv_dl)

{'val_loss': 25.529077529907227, 'val_acc': 0.3482142984867096}

In [156]:
# train: true, test: conv-inc
evaluate(model1, conv_inc_dl)

{'val_loss': 25.297374725341797, 'val_acc': 0.4073423147201538}

In [116]:
# train: conv, test: conv
evaluate(model2, conv_dl_test)

{'val_loss': 0.0, 'val_acc': 1.0}

In [117]:
# train: conv, test: true
evaluate(model2, true_dl)

{'val_loss': 55.46001052856445, 'val_acc': 0.5008548498153687}

In [ ]:
# train: conv-inc, test: conv-inc
evaluate(model3, conv_inc_dl_test)

In [76]:
# train: conv-inc, test: true
evaluate(model3, true_dl)

{'val_loss': 6.246950626373291, 'val_acc': 0.5075988173484802}